In [1]:
# 本代码由可视化策略环境自动生成 2022年1月21日 15:56
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m8_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    #context.set_commission(PerOrder(buy_cost=0.00001, sell_cost=0.0001, min_cost=1))
    
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    context.stock_count = 1
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    #context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 每只股票的权重平均分配
    context.stock_weights = 1/context.stock_count
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 1
    context.options['hold_days'] = 1


# 回测引擎：每日数据处理函数，每天执行一次
def m8_handle_data_bigquant_run(context, data):
    today = data.current_dt.strftime('%Y-%m-%d')
    equities = {e.symbol: p for e, p in context.portfolio.positions.items() if p.amount>0}
    stock_now = len(equities); #获取当前持仓股票数量
    stock_count = context.stock_count
    
    # 按日期过滤得到今日的预测数据
    # 加载预测数据
    df = context.options['data'].read_df()
    df_today = df[df.date == data.current_dt.strftime('%Y-%m-%d')]
    df_today.set_index('instrument')
    
    
    now_stock = []
    sell_stock = []
       
    try:
        buy_list = context.daily_buy_stock[today]
    except:
        buy_list = []

    
    # 1. 资金分配
    #is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天） 
    #stock_cash = context.portfolio.portfolio_value/stock_count
    #cash_avg = context.portfolio.portfolio_value
    #cash_for_buy = min(context.portfolio.cash,  stock_cash)
    #cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}
    
            
    #if not is_staging :
    if 1==1 :    
        if len(equities) > 0:
            for i in equities.keys():
                last_sale_date = equities[i].last_sale_date	# 上次交易日期
                delta_days = data.current_dt - last_sale_date  
                hold_days = delta_days.days # 持仓天数
                if hold_days >= context.options['hold_days'] and i not in buy_list :
                    print('日期:',today,'卖出2:',i)
                    context.order_target(context.symbol(i), 0)
                    sell_stock.append(i)
                    stock_now = stock_now -1
                    #print('日期：', today, '股票：', i, ' 卖出')
                 
# 3. 生成买入订单
    buy_num = stock_count - stock_now
    #if is_staging :
    #    buy_num = 1
    if len(buy_list)>0:
        print('日期：', today, '选出股票数量：', len(buy_list))
    if buy_num>0 and len(buy_list)>0 :
        # 不再买入已经轮仓卖出和移动止损的股票,以防止出现空头持仓
        buy_instruments = [i for i in buy_list if i not in now_stock][:buy_num]
        cash_for_buy = context.portfolio.cash/len(buy_instruments)
        for i, instrument in enumerate(buy_instruments):
            current_price = data.current(context.symbol(instrument), 'price')
            
            if cash_for_buy>0 and data.can_trade(context.symbol(instrument)):           
                amount = math.floor(cash_for_buy / current_price / 100) * 100
                context.order(context.symbol(instrument), amount)
                #if(instrument=='002735.SZA'):
                print('日期:',today,'买入:',instrument)
            else :
                print('日期:',today,'无资金或不能交易未买入:',instrument)
# 回测引擎：准备数据，只执行一次
def m8_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()
    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)
    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)
    
    # 每日卖出股票的数据框
    context.daily_sell_stock= df.groupby('date').apply(close_pos_con)  
    # 每日买入股票的数据框
    context.daily_buy_stock= df.groupby('date').apply(open_pos_con)  


    

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m8_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2020-01-01',
    end_date=T.live_run_param('trading_date', '2021-12-31'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
shouyi=(close_0-open_1)/open_1
amount_zf=amount_0/amount_1
amount_bl=amount_0/avg_amount_180
zgzzf=high_0/close_1
zhangf=(close_0-open_0)/close_1
dbzs_zhangf=(close_0-open_0)/open_0 - zs_zhangf
dbzs_return=return_0 - zs_return_0
zhangf_max=max((close_0-open_0)/open_0,(close_0-close_1)/close_1)
zhenf=(high_0-low_0)/close_1
return0=return_0
return1=shift(return_0,1)
return3=return_3
max10=ts_max(close_0,10)
max10d=ts_argmax(close_0,10)
min10=ts_min(close_0,10)
min10d=ts_argmin(close_0,10)
priceHighBl10=close_0/max10
priceLowBl10=close_0/min10
max30=ts_max(close_0,30)
max30d=ts_argmax(close_0,30)
min30=ts_min(close_0,30)
min30d=ts_argmin(close_0,30)
priceHighBl30=close_0/max30
priceLowBl30=close_0/min30
zs_huiluo
zs_return_0
zs_return_1
zs_priceHighBl10
zs_priceLowBl10
zs_priceHighBl30
zs_priceLowBl30
dbzs_priceHighBl10=priceHighBl10-zs_priceHighBl10
dbzs_priceLowBl10=priceLowBl10-zs_priceLowBl10
dbzs_priceHighBl30=priceHighBl30-zs_priceHighBl30
dbzs_priceLowBl30=priceLowBl30-zs_priceLowBl30
mf_net_pct_l_0
mf_net_pct_m_0
mf_net_pct_main_0
mf_net_pct_s_0
mf_net_pct_xl_0
open_0
zhangfMax5=ts_max(zhangf,5)
#当前10天最低价与前10天最低价比值
low10bl=min10/shift(min10,10)
#前10天的最低值与前20天的最高值比值
high20chu10bl=shift(max10,20)/shift(min10,10)
#当前价格与10天前，15日内的最低价的比值
closeOldLow25bl=close_0/shift(ts_min(close_0,15),10)
#10天前，15日内的最低价与25天前15天的最高价比值
closeOldLow25_15bl=shift(ts_min(close_0,15),25)/shift(ts_min(close_0,15),10)
ddzb=mf_net_pct_l_0+mf_net_pct_main_0+mf_net_pct_xl_0
dbzb30=sum(ddzb,30)
zlcy=ts_max(max(mf_net_pct_l_0,mf_net_pct_main_0,mf_net_pct_xl_0),5)
#前10天至40天的振幅
hpbl30_old=ts_max(shift(close_0,10),30)/ts_min(shift(close_0,10),30)
#最近10天的振幅
hpbl10=ts_max(close_0,10)/ts_min(close_0,10)
isHasZhangt=ts_max(where((return_0>1.09)&(close_0==high_0),1,0),10)
isHasDiet=ts_max(where((return_0<0.91)&(close_0==low_0),1,0),10)
mc1=where((zhangf_max>0.05)&(amount_zf<1),1,0)
mc2=where(close_0<mean(close_0,5),1,0)
mc3=where((return_0<1)&(close_0<open_0),1,0)
mc4=where((zhenf>0.08)&(return_0<1.025),1,0)
#连续下跌天数
isXiaDie0=where(return_0<1,1,0)
lxxd_1d=where(sum(isXiaDie0,1)==1,1,0)
lxxd_2d=where(sum(isXiaDie0,2)==2,2,0)
lxxd_3d=where(sum(isXiaDie0,3)==3,3,0)
lxxd_4d=where(sum(isXiaDie0,4)==4,4,0)
lxxd_5d=where(sum(isXiaDie0,5)==5,5,0)
lxxd_6d=where(sum(isXiaDie0,6)==6,6,0)
lxxd_8d=where(sum(isXiaDie0,8)==8,8,0)
lxxd_10d=where(sum(isXiaDie0,10)==10,10,0)
lxxd_days=max(lxxd_1d,lxxd_2d,lxxd_3d,lxxd_4d,lxxd_5d,lxxd_6d,lxxd_8d,lxxd_10d)
#连续上涨天数
isShangZ0=where(return_0>1,1,0)
lxsz_1d=where(sum(isShangZ0,1)==1,1,0)
lxsz_2d=where(sum(isShangZ0,2)==2,2,0)
lxsz_3d=where(sum(isShangZ0,3)==3,3,0)
lxsz_4d=where(sum(isShangZ0,4)==4,4,0)
lxsz_5d=where(sum(isShangZ0,5)==5,5,0)
lxsz_6d=where(sum(isShangZ0,6)==6,6,0)
lxsz_8d=where(sum(isShangZ0,8)==8,8,0)
lxsz_10d=where(sum(isShangZ0,10)==10,10,0)
lxsz_days=max(lxsz_1d,lxsz_2d,lxsz_3d,lxsz_4d,lxsz_5d,lxsz_6d,lxsz_8d,lxsz_10d)
jx5d=mean(close_0,5)
isLxsz=where((jx5d/shift(jx5d,5)>1)&(shift(jx5d,5)/shift(jx5d,10)>1),1,0)
#与值数涨幅的相关性
zszf_xgx_1=where((zs_return_0>1)&(return_0<1),1,0)
maxZhangf5=ts_max(zhangf,5)
maxZhangf5d=ts_argmax(zhangf,5)
maxZhangfOpen=where(maxZhangf5d>3,open_0,where(maxZhangf5d>2,shift(open_0,1),where(maxZhangf5d>1,shift(open_0,2),where(maxZhangf5d>0,shift(open_0,3),0))))
zhangf_low_bl=close_0/maxZhangfOpen
minZhangfLow=maxZhangfOpen/min(open_1,close_1,open_0)-1
#20年，胜率0.6，盈亏比2.91；18年3年胜率0.52，盈亏比1.94
my1=where((abs(return0-1.02)<0.015)&(priceHighBl10==1)&(priceLowBl10<1.08)&(abs(priceHighBl30-0.9)<0.05)&(abs(priceLowBl30-1.12)<0.05)&(amount_bl>3)&(amount_zf>1)&(dbzs_return>0)&(return0>return1)&((return3+zgzzf)<2.1),1,0)
#20年，胜率0.65，盈亏比2.51；18年3年，0.71，1.39，97支；
my3=where((return0>1)&(return1>1)&(zhangf>0)&(return3<0.97)&(priceHighBl10<0.88)&(priceLowBl10<1.07)&(priceHighBl30==priceHighBl10)&(priceLowBl30==priceLowBl10)&(dbzs_return>0)&(ddzb>0.05),1,0)
#18年3年，0.65胜，1.39盈亏比
my6=where((abs(return0-1.02)<0.01)&(return0>return1)&(lxsz_days==2)&((return0+return1)<2.04)&(return3<1)&(priceHighBl10<0.93)&(priceLowBl10<1.12)&(priceHighBl30==priceHighBl10)&(priceLowBl30==priceLowBl10)&(dbzs_return>0),1,0)
#18年3年，0.57胜，1.96盈亏比
my8=where((return0>return1)&(zhangf>0)&(zhenf<0.025)&(abs(return1-0.99)<0.005)&(priceLowBl10<1.05)&(abs(max10d-7)<3)&(abs(priceHighBl30-0.95)<0.05)&(priceLowBl30<1.15)&(zlcy>0.05)&(amount_bl>1.3)&(max30d<10),1,0)
#20年155支，胜率0.57，盈亏比1.76；18年3年，胜率0.5，盈亏比1.42
my11=where((lxsz_days>2)&(return3<1.06)&(dbzs_return>0)&(priceHighBl10==1)&(priceLowBl10<1.06)&(abs(priceHighBl30-0.9)<0.05)&(abs(priceLowBl30-priceLowBl10)<0.05)&(abs(dbzs_priceHighBl10-0.01)<0.01)&(ts_min(amount_zf,2)>1)&(ddzb>0.05),1,0)
#按策略优质度排序
yzd1=where(my3==1,10,0)
yzd2=where(my6==1,8,0)
yzd3=where(my8==1,6,0)
yzd4=where(my1==1,4,0)
yzd5=where(my11==1,2,0)
yzd=yzd1+yzd2+yzd3+yzd4+yzd5
my=max(my1,my3,my6,my8,my11)
buy_condition=where(my==1,1,0)
sell_condition=where(mc2==3,1,0)
#gltj=where((shouyi>0.08)&(shouyi<0.2)&(shift(dbzs_return,2)>0)&(shift(dbzs_priceLowBl10,2)<0.05)&(shift(dbzs_priceLowBl30,2)<1.15),1,0)"""
)

m5 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m2.data,
    start_date='',
    end_date='',
    before_start_days=60,
    m_cached=False
)

m10 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2050-12-30',
    market='CN_STOCK_A',
    instrument_list='000002.HIX',
    max_count=0
)

m12 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
zs_zhangf=(close-open)/open
zs_huiluo=(high-close)/close
zs_huishen=(close-low)/low
zs_zhenf=(high-low)/shift(close,1)
zs_volume_zf=volume/shift(volume,1)
zs_return_0=close/shift(close,1)
zs_return_1=shift(zs_return_0,1)
zs_return_2=shift(zs_return_0,2)
zs_max10=ts_max(close,10)
#zs_max10d=ts_argmax(close,10)
zs_max30=ts_max(close,30)
#zs_max30d=ts_argmax(close,30)
zs_min10=ts_min(close,10)
#zs_min10d=ts_argmin(close,10)
zs_min30=ts_min(close,30)
#zs_min30d=ts_argmin(close,30)
zs_priceHighBl10=close/zs_max10
zs_priceLowBl10=close/zs_min10
zs_priceHighBl30=close/zs_max30
zs_priceLowBl30=close/zs_min30"""
)

m4 = M.use_datasource.v1(
    instruments=m10.data,
    features=m12.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date=''
)

m13 = M.derived_feature_extractor.v3(
    input_data=m4.data,
    features=m12.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=True,
    user_functions={}
)

m18 = M.instruments.v2(
    start_date='2020-01-01',
    end_date='2020-12-30',
    market='CN_STOCK_A',
    instrument_list='000002.HIX',
    max_count=0
)

m19 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
"""
)

m17 = M.use_datasource.v1(
    instruments=m18.data,
    features=m19.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date=''
)

m20 = M.data_join.v3(
    input_1=m13.data,
    input_2=m17.data,
    on='date',
    how='left',
    sort=False
)

m14 = M.data_join.v3(
    input_1=m5.data,
    input_2=m20.data,
    on='date',
    how='left',
    sort=False
)

m7 = M.derived_feature_extractor.v3(
    input_data=m14.data,
    features=m2.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=True,
    user_functions={}
)

m6 = M.chinaa_stock_filter.v1(
    input_data=m7.data,
    index_constituent_cond=['全部'],
    board_cond=['上证主板', '深证主板'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m11 = M.dropnan.v2(
    input_data=m6.data
)

m9 = M.sort.v4(
    input_ds=m11.data,
    sort_by='yzd',
    group_by='date',
    keep_columns='--',
    ascending=False
)

m3 = M.filter.v3(
    input_data=m9.sorted_data,
    expr='my==1',
    output_left_data=False
)

m8 = M.trade.v4(
    instruments=m1.data,
    options_data=m3.data,
    start_date='',
    end_date='',
    initialize=m8_initialize_bigquant_run,
    handle_data=m8_handle_data_bigquant_run,
    prepare=m8_prepare_bigquant_run,
    before_trading_start=m8_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=100000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2022-01-21 15:48:55.557743] INFO: moduleinvoker: instruments.v2 开始运行..

[2022-01-21 15:48:55.598838] INFO: moduleinvoker: 命中缓存

[2022-01-21 15:48:55.600663] INFO: moduleinvoker: instruments.v2 运行完成[0.042939s].

[2022-01-21 15:48:55.609371] INFO: moduleinvoker: input_features.v1 开始运行..

[2022-01-21 15:48:55.618876] INFO: moduleinvoker: 命中缓存

...

[2022-01-21 15:52:50.630608] INFO: Performance: Simulated 486 trading days out of 486.

[2022-01-21 15:52:50.632358] INFO: Performance: first open: 2020-01-02 09:30:00+00:00

[2022-01-21 15:52:50.633468] INFO: Performance: last close: 2021-12-31 15:00:00+00:00

[2022-01-21 15:52:58.872819] INFO: moduleinvoker: backtest.v8 运行完成[33.614165s].

[2022-01-21 15:52:58.874430] INFO: moduleinvoker: trade.v4 运行完成[35.884717s].